In [1]:
import pytesseract
import cv2

In [13]:
img = cv2.imread("../datasets/Manga109/Manga109_released_2021_12_30/images/AisazuNihaIrarenai/005.jpg")

In [16]:
cv2.imshow("image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()